In [28]:
# load libararies

import os
import pandas as pd
import numpy as np
from pathlib import Path
from openpyxl import load_workbook

In [29]:
# a days difference function

def days_between(d1, d2):
    d1 = datetime.strptime(d1, "%Y-%m-%d")
    d2 = datetime.strptime(d2, "%Y-%m-%d")
    return abs((d2 - d1).days)

In [30]:
# convert files from .xls to .xlsx using MS Office

import win32com.client as win32
pth = r'C:\Users\hilton.netta\Downloads'
fPARN = pth + '\Portfolio Analytics Report - New.xls'
fDE = pth + '\Derivative Exposure.xls'

files = [fPARN, fDE]

for fl in files:
    excel = win32.gencache.EnsureDispatch('Excel.Application')
    wb = excel.Workbooks.Open(fl)
    wb.SaveAs(fl + 'x', FileFormat = 51)     #FileFormat = 51 is for .xlsx extension
    wb.Close()                               #FileFormat = 56 is for .xls extension

# excel.Application.Quit()

In [31]:
# set the names of the files to use

import datetime
from datetime import date
from datetime import timedelta

# variables ...
ydy = date.today() - timedelta(days=1)
yday = f'{ydy:%#d}{ydy:%#b}{ydy:%Y}' # current date without leading zeroes, e.g. 3May2021

# ... and paths
#pth = Path('//pim-cpt-statpro/SPC_602/Client/Profiles/PIM/Exports')
#flnDerv = pth / f'Derivative Exposure {yday}.xlsx'
#flnHoldings = pth / f'Portfolio Analytics Report - New {yday}.xlsx'
pth = Path('C:/Users/hilton.netta/Downloads')
flnDerv = pth / 'Derivative Exposure.xlsx'
flnHoldings = pth / 'Portfolio Analytics Report - New.xlsx'
flnHoldings

WindowsPath('C:/Users/hilton.netta/Downloads/Portfolio Analytics Report - New.xlsx')

In [32]:
# create dataframes of the derivatives and holdings spreadsheets

wbD = pd.read_excel(open(flnDerv,'rb'), sheet_name = 'Derivative Exposure')
wbH = pd.read_excel(open(flnHoldings,'rb'), sheet_name = 'Portfolio Analytics Report - Ne')
type(wbD), type(wbH)

(pandas.core.frame.DataFrame, pandas.core.frame.DataFrame)

In [33]:
# list the unique fund codes

fndcodes = wbH.loc[:,'Entity ID'].unique()
# print(fndcodes)

In [34]:
# find the NAV of each fund; this will be used as a lookup later

navs = wbH.groupby('Entity ID')['Sum of Market Value Income'].sum() # this has type 'pandas.core.series.Series'
# navs.head(3)

# convert the navs series to a dictionary to make it lookupable
nav = navs.to_dict()
print(navs.shape[0], len(navs), len(nav)) # .shape[0] doesnt work for dictionaries

77 77 77


In [35]:
# recalc the '% of Total Market Value' column per fund

newTMV=[]
for i, row in wbH.iterrows():
    if nav[row['Entity ID']] == 0:
        fndpct = 1
    else:
        fndpct = row['Sum of Market Value Income'] / nav[row['Entity ID']]
    
    newTMV.append(fndpct)
    
#type(newTMV)

In [37]:
# replace the '% of Total Market Value' with the values in the new list

# wbH['% of Total Market Value'] = newTMV

wbH['% of Total Market Value'].sum() # should equal number of funds

77.0

In [38]:
# save the dataframe as an Excel file

wbH.to_excel(pth / 'Portfolio Analytics Report - New.xlsx', index = False) # overwrites the previous file

In [41]:
# load the 'Derv.xlsm' workbook

# pathD = r'\\pim-cpt-fs\profile$\PProfile Operations\Legal\Legal and Compliance Framework\Hilton\W\Derv.xlsm'
pathD = Path('//pim-cpt-fs/profile$/PProfile Operations/Legal/Legal and Compliance Framework/Hilton/W/Derv.xlsm')
wbDerv = load_workbook(pathD)

In [110]:
writer = pd.ExcelWriter(pathD, engine = 'openpyxl')
writer.book = wbDerv